# DiTTo

DiTTo is the Distribution Transformation Tool. It is an open source tool to convert and modify electrical distribution system models. DiTTo implements a many-to-one-to-many parsing framework, making it modular and robust. The reader modules parse data files of distribution system format (e.g. OpenDSS) and create an object for each electrical component. These objects are stored in a GDM DistributionSystem instance. The writer modules are then used to export the data stored in memory to a selected output distribution system format (e.g. OpenDSS) which are written to disk. 

For model conversion. We will need to install DiTto. To do that, use the command below

In [ ]:
pip install NREL-ditto opendssdirect.py 

Note: you may need to restart the kernel to use updated packages.


We start by loading a sample GDM system using the gdmloader.

In [1]:
from gdm.distribution import DistributionSystem
from gdmloader.constants import GCS_CASE_SOURCE
from gdmloader.source import SystemLoader

gdm_loader = SystemLoader()
gdm_loader.add_source(GCS_CASE_SOURCE)

distribution_system: DistributionSystem = gdm_loader.load_dataset(
    source_name=GCS_CASE_SOURCE.name, 
    system_type=DistributionSystem, 
    dataset_name="p5r",
)
distribution_system.name = "P5R"

/opt/homebrew/Caskroom/miniconda/base/envs/gdm2/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


Once installed, a GDM model can be loaded into memory and converted to any of the supported formats using one of ditto writers

In [2]:
from pathlib import Path
from ditto.writers.opendss.write import Writer


writer = Writer(distribution_system)
writer.write(output_path=Path("."), separate_substations=False, separate_feeders=False)

2025-05-02 11:06:28.929 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper VoltageLimitSetMapper not found. Skipping
2025-05-02 11:06:28.929 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper LocationMapper not found. Skipping
2025-05-02 11:06:28.929 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper PhaseVoltageSourceEquipmentMapper not found. Skipping
2025-05-02 11:06:28.930 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper PhaseLoadEquipmentMapper not found. Skipping
2025-05-02 11:06:28.930 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper InverterEquipmentMapper not found. Skipping
2025-05-02 11:06:28.930 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper SolarEquipmentMapper not found. Skipping
2025-05-02 11:06:28.930 | WARNING  | ditto.writers.opendss.write:write:80 - Mapper WindingEquipmentMapper not found. Skipping
2025-05-02 11:06:28.931 | DEBUG    | ditto.writers.opendss.write:write:83 - Mapping components in MatrixImpedanceSw


In this example, the OpenDSS model contains intentional errors such as missing bus definitions, incorrect syntax for loads, and incorrect capacitor connections. The Python script uses GDM's `GDMReader` to attempt to read the model, which should identify and report these issues.


In [3]:

from ditto.readers.opendss.reader import Reader
from opendssdirect import dss

dss("""
    clear
    new Circuit.test bus1=bus_1.1 BasekV=7.2 pu=1.03 Angle=0.0 Phases=1 Z1=[1e-05, 1e-05] Z0=[1e-05, 1e-05]
    new line.line_1  bus1=bus_1.1 bus2=bus_2.2 phases=1
    solve
""")
dss.Circuit.Save("test_circuit", options=0)

reader = Reader("test_circuit/Master.dss")
gdm_system = reader.get_system()


2025-05-02 11:06:29.176 | DEBUG    | ditto.readers.opendss.reader:_read:61 - Loading OpenDSS model.
2025-05-02 11:06:29.180 | DEBUG    | ditto.readers.opendss.reader:_read:70 - Model loaded from test_circuit/Master.dss.
2025-05-02 11:06:29.181 | DEBUG    | ditto.readers.opendss.components.buses:get_buses:22 - parsing bus components...
2025-05-02 11:06:29.182 | DEBUG    | ditto.readers.opendss.components.sources:get_voltage_sources:88 - parsing voltage sources components...
2025-05-02 11:06:29.183 | DEBUG    | ditto.readers.opendss.components.loadshapes:build_profiles:103 - parsing timeseries components...
2025-05-02 11:06:29.183 | DEBUG    | ditto.readers.opendss.components.capacitors:get_capacitors:89 - parsing capacitor components...
2025-05-02 11:06:29.184 | DEBUG    | ditto.readers.opendss.components.loads:get_loads:95 - parsing load components...
2025-05-02 11:06:29.184 | DEBUG    | ditto.readers.opendss.components.pv_systems:get_pvsystems:83 - parsing pvsystem components...
2025-

ValidationError: 1 validation error for MatrixImpedanceBranch
  Value error, Conductor phase (phase=<Phase.A: 'A'>) does not match bus phases (bus.phases=[<Phase.B: 'B'>]) [type=value_error, input_value={'name': 'line_1', 'buses...tity(400.0, 'ampere')>)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [4]:

from ditto.readers.opendss.reader import Reader
from opendssdirect import dss

dss("""
    clear
    new Circuit.test bus1=bus_1.1 BasekV=7.2 pu=1.03 Angle=0.0 Phases=1 Z1=[1e-05, 1e-05] Z0=[1e-05, 1e-05]
    new line.line_1  bus1=bus_1.1 bus2=bus_2.1 phases=1 length=-10
    solve
""")
dss.Circuit.Save("test_circuit", options=0)

reader = Reader("test_circuit/Master.dss")
gdm_system = reader.get_system()

2025-05-02 11:07:47.071 | DEBUG    | ditto.readers.opendss.reader:_read:61 - Loading OpenDSS model.
2025-05-02 11:07:47.075 | DEBUG    | ditto.readers.opendss.reader:_read:70 - Model loaded from test_circuit/Master.dss.
2025-05-02 11:07:47.076 | DEBUG    | ditto.readers.opendss.components.buses:get_buses:22 - parsing bus components...
2025-05-02 11:07:47.077 | DEBUG    | ditto.readers.opendss.components.sources:get_voltage_sources:88 - parsing voltage sources components...
2025-05-02 11:07:47.078 | DEBUG    | ditto.readers.opendss.components.loadshapes:build_profiles:103 - parsing timeseries components...
2025-05-02 11:07:47.078 | DEBUG    | ditto.readers.opendss.components.capacitors:get_capacitors:89 - parsing capacitor components...
2025-05-02 11:07:47.078 | DEBUG    | ditto.readers.opendss.components.loads:get_loads:95 - parsing load components...
2025-05-02 11:07:47.079 | DEBUG    | ditto.readers.opendss.components.pv_systems:get_pvsystems:83 - parsing pvsystem components...
2025-

AssertionError: Distance (-10.0, m) must be positive.

In [ ]:
from ditto.readers.opendss.reader import Reader
from opendssdirect import dss

dss("""
clear
new Circuit.test bus1=bus_1.1 BasekV=7.2 pu=1.03 Angle=0.0 Phases=1 Z1=[1e-05, 1e-05] Z0=[1e-05, 1e-05]
new line.line_1  bus1=bus_1.1 bus2=bus_2.1 phases=1
new load.load_1  bus=bus_2.1.2.3 phases=1
solve
""")
dss.Circuit.Save("test_circuit", options=0)

reader = Reader("test_circuit/Master.dss")


/opt/homebrew/Caskroom/miniconda/base/envs/gdm2/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
2025-05-01 15:23:51.732 | DEBUG    | ditto.readers.opendss.reader:_read:61 - Loading OpenDSS model.
2025-05-01 15:23:51.737 | DEBUG    | ditto.readers.opendss.reader:_read:70 - Model loaded from test_circuit/Master.dss.
2025-05-01 15:23:51.738 | DEBUG    | ditto.readers.opendss.components.buses:get_buses:22 - parsing bus components...
2025-05-01 15:23:51.739 | DEBUG    | ditto.readers.opendss.components.sources:get_voltage_sources:88 - parsing voltage sources components...
2025-05-01 15:23:51.740 | DEBUG    | ditto.readers.opendss.components.loadshapes:build_profiles:103 - parsing tim

ValidationError: 1 validation error for DistributionLoad
  Value error, Loads phases self.phases=[<Phase.A: 'A'>, <Phase.B: 'B'>, <Phase.C: 'C'>] must be subset of bus phases. [<Phase.A: 'A'>, <Phase.B: 'B'>] [type=value_error, input_value={'name': 'load_1', 'bus':...tionType.STAR: 'STAR'>)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

## Validation Summary Via Monkey Patching

In [5]:
from pydantic import ValidationError
from rich.console import Console
from infrasys import Component
from rich.table import Table

validation_errors = []
_original_init = Component.__init__
def patched_init(self, *args, **kwargs):
    try:
        _original_init(self, *args, **kwargs)
    except ValidationError as e:
        for error in e.errors():   
            validation_errors.append([
                kwargs['name'],
                self.__class__.__name__,
                error['loc'] if error['loc'] else "On model validation",
                error['type'],
                error['msg']
            ]) 


Component.__init__ = patched_init

from ditto.readers.opendss.reader import Reader
from opendssdirect import dss

dss("""
    clear
    new Circuit.test bus1=bus_1.1 BasekV=7.2 pu=1.03 Angle=0.0 Phases=1 Z1=[1e-05, 1e-05] Z0=[1e-05, 1e-05]
    new line.line_1  bus1=bus_1.1 bus2=bus_2.2 phases=3
    solve
""")
dss.Circuit.Save("test_circuit", options=0)

reader = Reader("test_circuit/Master.dss")

if validation_errors:
    error_table = Table(title="Validation warning summary")
    error_table.add_column("Model", justify="right", style="cyan", no_wrap=True)
    error_table.add_column("Type", style="green")
    error_table.add_column("Field", justify="right", style="bright_magenta")
    error_table.add_column("Error", style="bright_red")
    error_table.add_column("Message", justify="right", style="turquoise2")

    for row in validation_errors:
        error_table.add_row(*row)
    
    console = Console()
    console.print(error_table)

    raise Exception(f"Validations errors occured when running the script. See the table above")


2025-05-02 11:07:58.868 | DEBUG    | ditto.readers.opendss.reader:_read:61 - Loading OpenDSS model.
2025-05-02 11:07:58.873 | DEBUG    | ditto.readers.opendss.reader:_read:70 - Model loaded from test_circuit/Master.dss.
2025-05-02 11:07:58.873 | DEBUG    | ditto.readers.opendss.components.buses:get_buses:22 - parsing bus components...
2025-05-02 11:07:58.875 | DEBUG    | ditto.readers.opendss.components.sources:get_voltage_sources:88 - parsing voltage sources components...
2025-05-02 11:07:58.875 | DEBUG    | ditto.readers.opendss.components.loadshapes:build_profiles:103 - parsing timeseries components...
2025-05-02 11:07:58.877 | DEBUG    | ditto.readers.opendss.components.capacitors:get_capacitors:89 - parsing capacitor components...
2025-05-02 11:07:58.877 | DEBUG    | ditto.readers.opendss.components.loads:get_loads:95 - parsing load components...
2025-05-02 11:07:58.877 | DEBUG    | ditto.readers.opendss.components.pv_systems:get_pvsystems:83 - parsing pvsystem components...
2025-

System                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Property             ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ System name          │       │
│ Data format version  │ 2.0.0 │
│ Components attached  │    10 │
│ Time Series attached │     0 │
│ Description          │       │
└──────────────────────┴───────┘

Component Information                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                           ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ DistributionBus                │     2 │
│ DistributionVoltageSource      │     1 │
│ Location                       │     1 │
│ MatrixImpedanceBranch          │     1 │
│ MatrixImpedanceBranchEquipment │     1 │
│ PhaseVoltageSourceEquipment    │     1 │
│ VoltageLimitSet                │     2 │
│ VoltageSourceEquipment         │     1 │
└────────────────────────────────┴───────┘

2025-05-02 11:07:58.900 | DEBUG    | ditto.readers.opendss.reader:_read:109 - 
None
2025-05-02 11:07:58.901 | DEBUG    | ditto.readers.opendss.reader:_read:110 - Building graph...
2025-05-02 11:07:58.901 | DEBUG    | ditto.readers.opendss.reader:_read:112 - Graph with 2 nodes and 1 edges
2025-05-02 11:07:58.902 | DEBUG    | ditto.readers.opendss.reader:_read:113 - Graph build complete...
2025-05-02 11:07:58.902 | DEBUG    | ditto.readers.opendss.reader:_read:114 - Updating graph to fix split phase representation...
2025-05-02 11:07:58.903 | DEBUG    | ditto.readers.opendss.reader:_read:116 - System update complete...


                                            Validation warning summary                                             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Model ┃ Type                  ┃               Field ┃ Error       ┃                                    Message ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ line_1 │ MatrixImpedanceBranch │ On model validation │ value_error │               Value error, Conductor phase │
│        │                       │                     │             │  (phase=<Phase.A: 'A'>) does not match bus │
│        │                       │                     │             │        phases (bus.phases=[<Phase.B: 'B'>, │
│        │                       │                     │             │                           <Phase.C: 'C'>]) │
└────────┴───────────────────────┴─────────────────────┴─────────────┴────────────────────────────────────────────┘

Exception: Validations errors occured when running the script. See the table above